In [2]:
import geopandas as gpd
from shapely.geometry import box
import numpy as np

In [47]:
def recursive_partition(w, h, min):
    if min < 0.01: raise ValueError("Minimum bbox area must be at least 0.01 metres")
    new_w = w / 2
    new_h = h / 2
    if new_w * new_h < min:
        return w, h
    return recursive_partition(new_w, new_h, min)

def log_partition(w, h, min):
    splits = np.floor(0.5 * np.log2(w * h / min))
    return w / 2**splits, h / 2**splits

def split_bbox(bbox, min_area, partition_func):
    bounds = bbox.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    w, h = partition_func(width, height, min_area)

    bboxes = []
    for i in range(int(width / w)):
        for j in range(int(height / h)):
            b = box(bounds[0] + i * w, bounds[1] + j * h, bounds[0] + (i+1) * w, bounds[1] + (j+1) * h)
            bboxes.append(b)
    return bboxes

bbox = box(0, 0, 16, 12)
bboxes = split_bbox(bbox, 0.5, recursive_partition)
gdf = gpd.GeoDataFrame({"geometry": bboxes})
gdf.explore()